# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import hvplot.pandas
import bokeh

# Import API keys
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,luderitz,-26.6481,15.1594,17.18,87,13,4.14,NaN,1667793929
1,1,saint george,37.1041,-113.5841,20.24,28,0,5.66,US,1667829893
2,2,port alfred,-33.5906,26.8910,18.43,90,100,1.79,ZA,1667790436
3,3,hithadhoo,-0.6000,73.0833,27.69,78,87,7.79,MV,1667782031
4,4,coquimbo,-29.9533,-71.3436,15.85,77,100,4.63,CL,1667814370


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    xlabel = "Longitude",
    ylabel = "Latitude",
    title = "City Data",
    legend = False,
    alpha=0.5,
    size = "Humidity",
    color="City"
    )

# Display the map plot
city_data_map

TypeError: '<' not supported between instances of 'Version' and 'str'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values

# Set ideal temp
ideal_temp = (city_data_df["Max Temp"] > 18) & (city_data_df["Max Temp"] < 24)

# Set ideal wind speed
ideal_wind = (city_data_df["Wind Speed"] < 5)

# Set ideal cloudiness %
ideal_cloud = (city_data_df["Cloudiness"] <= 10)

# Narrow down dataframe to ideal conditions
city_data_df = city_data_df[ideal_temp & ideal_wind & ideal_cloud]

# Drop any rows with null values
city_data_df.dropna(how='any', inplace=True)

# Display sample data
city_data_df

/var/folders/f5/fmwr4j4s4gv2xp3_q6xpt02w0000gn/T/ipykernel_4903/3567996527.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df.dropna(how='any', inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,albany,42.6001,-73.9662,21.33,36,0,0.89,US,1667821030
32,32,alice springs,-23.7000,133.8833,21.75,20,8,2.06,AU,1667852076
38,38,cape town,-33.9258,18.4232,19.23,70,0,4.63,ZA,1667792431
56,56,qesarya,32.5189,34.9046,21.96,57,0,2.68,IL,1667793760
152,152,lima,-12.0432,-77.0282,18.14,68,0,4.63,PE,1667817236
166,166,bin qirdan,33.1378,11.2197,20.27,49,0,3.24,TN,1667799508
250,250,east london,-33.0153,27.9116,21.52,94,0,3.60,ZA,1667790252
267,267,general cepeda,25.3833,-101.4500,23.93,31,0,2.50,MX,1667825857
302,302,tupancireta,-29.0806,-53.8358,22.50,41,1,4.63,BR,1667810254
367,367,townsville,-19.2500,146.8000,22.49,78,8,2.57,AU,1667849351


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,albany,US,42.6001,-73.9662,36,
32,alice springs,AU,-23.7000,133.8833,20,
38,cape town,ZA,-33.9258,18.4232,70,
56,qesarya,IL,32.5189,34.9046,57,
152,lima,PE,-12.0432,-77.0282,68,
166,bin qirdan,TN,33.1378,11.2197,49,
250,east london,ZA,-33.0153,27.9116,94,
267,general cepeda,MX,25.3833,-101.4500,31,
302,tupancireta,BR,-29.0806,-53.8358,41,
367,townsville,AU,-19.2500,146.8000,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
cape town - nearest hotel: Townhouse Hotel
qesarya - nearest hotel: דן קיסריה
lima - nearest hotel: Hotel Peach Garden
bin qirdan - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
general cepeda - nearest hotel: Casa del maestro
tupancireta - nearest hotel: Hotel Cassino
townsville - nearest hotel: Aquarius on the Beach
saint-pierre - nearest hotel: Tropic Hotel
victor harbor - nearest hotel: Hotel Victor
tura - nearest hotel: No hotel found
radcliff - nearest hotel: Gold Vault Inn
rampura - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
23,albany,US,42.6001,-73.9662,36,No hotel found
32,alice springs,AU,-23.7000,133.8833,20,Aurora Alice Springs
38,cape town,ZA,-33.9258,18.4232,70,Townhouse Hotel
56,qesarya,IL,32.5189,34.9046,57,דן קיסריה
152,lima,PE,-12.0432,-77.0282,68,Hotel Peach Garden
166,bin qirdan,TN,33.1378,11.2197,49,No hotel found
250,east london,ZA,-33.0153,27.9116,94,No hotel found
267,general cepeda,MX,25.3833,-101.4500,31,Casa del maestro
302,tupancireta,BR,-29.0806,-53.8358,41,Hotel Cassino
367,townsville,AU,-19.2500,146.8000,78,Aquarius on the Beach


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

hotel_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 1,
    xlabel = "Longitude",
    ylabel = "Latitude",
    title = "Hotel Map",
    legend = False,
    alpha=0.5,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
    )

    
# Display the map
hotel_data_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)